In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# URL do dataset HateBR no GitHub
DATASET_URL = 'https://raw.githubusercontent.com/franciellevargas/HateBR/main/dataset/HateBR.csv'

# 1. Carregar o dataset
print("Carregando o dataset...")
try:
    df = pd.read_csv(DATASET_URL)
    print("Dataset carregado com sucesso!")
    # Mostra as primeiras linhas e a distribuição das classes
    print(df.head())
    print("\nDistribuição das classes:")
    print(df['label_final'].value_counts(normalize=True))
except Exception as e:
    print(f"Erro ao carregar o dataset: {e}")
    exit()

# 2. Pré-processamento e Definição das variáveis
print("\nIniciando pré-processamento...")
# Para este MVP, a única limpeza será converter para minúsculas.
# O TfidfVectorizer já lida com muita coisa.
X = df['comentario'].str.lower()
y = df['label_final']

# 3. Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Dados divididos: {len(X_train)} para treino, {len(X_test)} para teste.")

# 4. Vetorização do texto usando TF-IDF
print("Vetorizando o texto...")
vectorizer = TfidfVectorizer(max_features=5000) # Usamos as 5000 palavras mais relevantes

# Aprende o vocabulário com os dados de treino e transforma os dados de treino
X_train_vect = vectorizer.fit_transform(X_train)

# Apenas transforma os dados de teste com o vocabulário já aprendido
X_test_vect = vectorizer.transform(X_test)

# 5. Treinamento do modelo de Regressão Logística
print("Treinando o modelo de classificação...")
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vect, y_train)
print("Modelo treinado com sucesso!")

# 6. Avaliação do modelo
print("\nAvaliando o modelo nos dados de teste...")
y_pred = model.predict(X_test_vect)

print("\nAcurácia:", accuracy_score(y_test, y_pred))
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=['Não Odioso', 'Odioso']))

# Agora que o modelo está treinado, podemos usá-lo.
# Os objetos que precisamos salvar/usar para novas previsões são:
# - `model` (o classificador)
# - `vectorizer` (o vetorizador)

print("\n--- MVP PRONTO PARA USO ---")

Carregando o dataset...
Dataset carregado com sucesso!
   id                                         comentario  anotator1  \
0   1                                       Mais um lixo          1   
1   2                    Essa nao tem vergonha na cara!!          1   
2   3                     Essa mulher é doente.pilantra!          1   
3   4                                Comunista safada...          1   
4   5  Vagabunda. Comunista. Mentirosa. O povo chilen...          1   

   anotator2  anotator3  label_final  \
0          1          1            1   
1          1          1            1   
2          1          1            1   
3          1          1            1   
4          1          1            1   

                                 links_post    account_post  
0  https://www.instagram.com/p/B2uThqdH9xI/  Carla Zambelli  
1  https://www.instagram.com/p/B2uThqdH9xI/  Carla Zambelli  
2  https://www.instagram.com/p/B2uThqdH9xI/  Carla Zambelli  
3  https://www.instagram.com/

In [62]:
def avaliar_toxicidade(comentario: str, model, vectorizer) -> dict:
    """
    Recebe um comentário e retorna a classificação de toxicidade
    e a probabilidade de ser discurso de ódio.
    """
    # 1. Aplicar o mesmo pré-processamento (minúsculas)
    comentario_processado = comentario.lower()
    
    # 2. Vetorizar o comentário usando o vetorizador JÁ TREINADO
    comentario_vect = vectorizer.transform([comentario_processado])
    
    # 3. Fazer a predição
    predicao = model.predict(comentario_vect)
    probabilidades = model.predict_proba(comentario_vect)
    
    # A probabilidade de ser discurso de ódio é a probabilidade da classe "1"
    prob_odio = probabilidades[0][1]
    
    if predicao[0] == 1:
        classificacao = "Discurso de Ódio"
    else:
        classificacao = "Não é Discurso de Ódio"
        
    return {
        "classificacao": classificacao,
        "nivel_toxicidade": f"{prob_odio:.2%}" # Formata como porcentagem
    }

In [63]:
# --- EXEMPLOS DE USO ---
print("\n--- Testando o MVP com novos comentários ---")

# Exemplo 1: Comentário potencialmente tóxico
comentario1 = "Esses políticos são todos uns bandidos, tinham que sumir do mapa!"
resultado1 = avaliar_toxicidade(comentario1, model, vectorizer)
print(f"Comentário: '{comentario1}'")
print(f"Resultado: {resultado1}\n")

# Exemplo 2: Comentário neutro
comentario2 = "O jogo de futebol ontem foi muito emocionante, gostei bastante do resultado."
resultado2 = avaliar_toxicidade(comentario2, model, vectorizer)
print(f"Comentário: '{comentario2}'")
print(f"Resultado: {resultado2}\n")


--- Testando o MVP com novos comentários ---
Comentário: 'Esses políticos são todos uns bandidos, tinham que sumir do mapa!'
Resultado: {'classificacao': 'Discurso de Ódio', 'nivel_toxicidade': '88.90%'}

Comentário: 'O jogo de futebol ontem foi muito emocionante, gostei bastante do resultado.'
Resultado: {'classificacao': 'Não é Discurso de Ódio', 'nivel_toxicidade': '35.25%'}



In [64]:
# (Após o bloco de treinamento e avaliação do modelo...)
import joblib

print("\nSalvando o modelo e o vetorizador em arquivos...")

# Salva o modelo treinado
joblib.dump(model, 'modelo_odio.joblib')

# Salva o vetorizador
joblib.dump(vectorizer, 'vetorizador_odio.joblib')

print("Modelo e vetorizador salvos com sucesso!")


Salvando o modelo e o vetorizador em arquivos...
Modelo e vetorizador salvos com sucesso!


In [66]:
import praw
import joblib
from dotenv import load_dotenv
import os

# --- CONFIGURAÇÃO ---

load_dotenv()

CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
USER_AGENT = os.getenv("USER_AGENT")

# Carregar o modelo e o vetorizador salvos
try:
    model = joblib.load('modelo_odio.joblib')
    vectorizer = joblib.load('vetorizador_odio.joblib')
    print("Modelo e vetorizador carregados com sucesso.")
except FileNotFoundError:
    print("ERRO: Arquivos 'modelo_odio.joblib' ou 'vetorizador_odio.joblib' não encontrados.")
    print("Certifique-se de executar o script de treinamento ('analisador_odio.py') primeiro.")
    exit()

# --- FUNÇÕES ---

def avaliar_toxicidade(comentario: str) -> dict:
    """
    Recebe um comentário e retorna a classificação de toxicidade
    e a probabilidade de ser discurso de ódio.
    """
    comentario_processado = comentario.lower()
    comentario_vect = vectorizer.transform([comentario_processado])
    predicao = model.predict(comentario_vect)
    probabilidades = model.predict_proba(comentario_vect)
    prob_odio = probabilidades[0][1]
    
    return {
        "eh_odio": predicao[0] == 1,
        "nivel_toxicidade": prob_odio
    }

# --- EXECUÇÃO PRINCIPAL ---

def main():
    print("\n--- INICIANDO ANÁLISE DE TOXICIDADE NO REDDIT ---")

    # Conectar à API do Reddit
    try:
        reddit = praw.Reddit(
            client_id=CLIENT_ID,
            client_secret=CLIENT_SECRET,
            user_agent=USER_AGENT,
            check_for_async=False
        )
        print(f"Conexão com a API do Reddit estabelecida. Modo Read-Only: {reddit.read_only}")
    except Exception as e:
        print(f"Erro ao conectar com a API do Reddit: {e}")
        return

    subreddit_alvo = "brasil"
    limite_comentarios = 100

    print(f"\nBuscando posts populares no r/{subreddit_alvo}...")
    
    subreddit = reddit.subreddit(subreddit_alvo)
    try:
        post_popular = next(p for p in subreddit.hot(limit=5) if not p.stickied)
        print(f"Analisando comentários do post: '{post_popular.title}'")
    except StopIteration:
        print(f"Não foi possível encontrar um post válido no r/{subreddit_alvo}.")
        return
    except Exception as e:
        print(f"Ocorreu um erro ao buscar o post: {e}")
        return

    total_toxicidade = 0
    comentarios_odiosos = 0
    comentarios_analisados = 0
    
    # NOVO: Lista para armazenar os comentários classificados como ódio
    comentarios_odiosos_lista = []

    post_popular.comments.replace_more(limit=0)

    for comment in post_popular.comments.list():
        if comentarios_analisados >= limite_comentarios:
            break
        
        texto_comentario = comment.body
        
        if not texto_comentario or texto_comentario in ['[deleted]', '[removed]']:
            continue

        resultado = avaliar_toxicidade(texto_comentario)
        total_toxicidade += resultado["nivel_toxicidade"]
        
        if resultado["eh_odio"]:
            comentarios_odiosos += 1
            # NOVO: Adiciona o texto do comentário à lista
            comentarios_odiosos_lista.append(texto_comentario)
        
        comentarios_analisados += 1

    media_toxicidade = (total_toxicidade / comentarios_analisados) if comentarios_analisados > 0 else 0
    
    print("\n--- Relatório Final da Análise ---")
    print(f"  - Subreddit analisado: r/{subreddit_alvo}")
    print(f"  - Post: '{post_popular.title[:60]}...'")
    print(f"  - Comentários analisados: {comentarios_analisados}")
    print(f"  - Comentários classificados como discurso de ódio: {comentarios_odiosos}")
    print(f"  - Nível médio de toxicidade nos comentários: {media_toxicidade:.2%}")

    # NOVO: Loop para imprimir os comentários ofensivos encontrados
    print("\n" + "="*50) # Adiciona uma linha separadora

    if comentarios_odiosos_lista:
        print("\n--- Comentários Classificados como Discurso de Ódio ---")
        for i, comentario in enumerate(comentarios_odiosos_lista, 1):
            print(f"{i}. {comentario}\n---") # Adiciona um separador entre os comentários
    else:
        print("\nNenhum comentário foi classificado como discurso de ódio nesta amostra.")


if __name__ == "__main__":
    main()

Modelo e vetorizador carregados com sucesso.

--- INICIANDO ANÁLISE DE TOXICIDADE NO REDDIT ---
Conexão com a API do Reddit estabelecida. Modo Read-Only: True

Buscando posts populares no r/brasil...
Analisando comentários do post: 'Deputado Nikolas Ferreira recebendo o adesivo do DIVINO FUSCA CLUBE'

--- Relatório Final da Análise ---
  - Subreddit analisado: r/brasil
  - Post: 'Deputado Nikolas Ferreira recebendo o adesivo do DIVINO FUSC...'
  - Comentários analisados: 100
  - Comentários classificados como discurso de ódio: 55
  - Nível médio de toxicidade nos comentários: 53.29%


--- Comentários Classificados como Discurso de Ódio ---
1. MDS essa porra de águia. Kkkkkkkkk
---
2. Fomos de apito de cachorro pra corneta de cachorro muito rápido
---
3. Que lamentável ver o James Cameron ao lado desse verme, espero que Avatar 3 flope
---
4. Esse adesivo não vai ter como negar que é nazismo velado. Vão ter que usar a desculpa do “não sabia” ou “não vi”.
---
5. Clube de carro antigo e de